<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - KNN User User nDCG </h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, user_knn as knn
from lenskit import topn
import pandas as pd
import numpy as np

ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [2]:
def model_trainer(neighbours,center):
    neighbours=int(neighbours)
    all_recs = []
    test_data = []
    algo_uu = knn.UserUser(neighbours,center=center,aggregate='weighted-average')
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test)
        all_recs.append(eval('UserUser', algo_uu, train, test))
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla = topn.RecListAnalysis()
    rla.add_metric(topn.ndcg)
    results = rla.compute(all_recs, test_data)
    return results['ndcg'].mean()


In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    neighbours = params['neighbours']
    center = params['center']
    metric = model_trainer(neighbours,center)
    print([neighbours,center,metric],',')
    return {'loss': -metric, 'status': STATUS_OK }

space= {'neighbours': hp.uniform('neighbours', 1, 100),
       'center': hp.choice('center', ['True', 'False'])}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

[46.22388452656476, 'True', 0.06217254410613936]                                                                       
,                                                                                                                      
[81.41945351172295, 'False', 0.05766441139867941]                                                                      
,                                                                                                                      
[38.52937811736078, 'True', 0.06293668854332474]                                                                       
,                                                                                                                      
[90.85992232095545, 'False', 0.05937203500712275]                                                                      
,                                                                                                                      
[24.18987487463559, 'False', 0.064218554

[16.07493631155823, 'True', 0.06827207802102114]                                                                       
,                                                                                                                      
[9.212279119782192, 'True', 0.06588904907249821]                                                                       
,                                                                                                                      
[95.01175383939986, 'True', 0.056720156458956165]                                                                      
,                                                                                                                      
[36.202572573314555, 'True', 0.06238433020641075]                                                                      
,                                                                                                                      
[25.64270327114102, 'True', 0.0632632850

[15.198726181920414, 'True', 0.06538181060331227]                                                                      
,                                                                                                                      
[37.97128486288275, 'True', 0.06356251139495088]                                                                       
,                                                                                                                      
[4.197112448742941, 'True', 0.06813533137734634]                                                                       
,                                                                                                                      
[10.134563712195922, 'True', 0.0673533069609664]                                                                       
,                                                                                                                      
[17.83740418783816, 'True', 0.0642601472

In [6]:
metric = pd.DataFrame(array)
metric.columns = ['neighbours','center','nDCG']
metric

,neighbours,center,nDCG
0,46.223885,True,0.062173
1,81.419454,False,0.057664
2,38.529378,True,0.062937
3,90.859922,False,0.059372
4,24.189875,False,0.064219
...,...,...,...
95,31.637694,True,0.063055
96,20.354704,True,0.063865
97,26.656630,True,0.062960
98,80.278112,False,0.058318


In [7]:
metric.sort_values(by=['nDCG'],ascending=False).head(10)

,neighbours,center,nDCG
26,3.125492,True,0.071334
12,4.824426,True,0.070431
62,4.851090,True,0.069663
67,5.160752,True,0.069478
89,6.641601,False,0.069263
85,3.176047,True,0.069142
32,2.797998,True,0.068451
66,6.948356,True,0.068428
24,5.377789,True,0.068371
34,16.074936,True,0.068272


In [8]:
metric.to_csv('metric_nDCG.csv', index=False)